<a href="https://colab.research.google.com/github/vahid-am/Vahid-am.github.io/blob/main/Fraud_GANs_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fraud Detection with GAN and Random Forest

---

## Objective
This notebook addresses class imbalance in fraud detection using:
1. **GANs** to generate synthetic minority class samples.
2. **Random Forest Classifier** to evaluate performance with GAN-augmented data.
3. **Evaluation Metrics** (precision, recall, F1-score) to assess model improvements.

---

## Dataset
- **Source**: 'creditcard.csv'
- **Features**: 'Time', 'Amount', 'V1' to 'V28' (PCA-transformed features).
- **Target ('Class')**:
  - '0': Non-fraudulent
  - '1': Fraudulent

---

## Workflow

1. **Generate Synthetic Data**:
   - Train a GAN to create realistic synthetic samples for the minority class.
   - Combine real and synthetic data to form a balanced dataset.

2. **Baseline Evaluation**:
   - Use Random Oversampling and train a Random Forest classifier.
   - Evaluate performance on the original test set.

3. **Reload Balanced Dataset**:
   - Verify class balance and prepare the data for modeling.


4. **GAN-Augmented Evaluation**:
   - Retrain the Random Forest with the GAN-balanced dataset.
   - Compare metrics against the baseline.

5. **Performance Insights**:
   - Assess the impact of GANs in improving fraud detection.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model, Sequential
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    precision_score,
    recall_score,
    f1_score
)
from imblearn.over_sampling import RandomOverSampler
from utils import GANDataBalancer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [ ]:
Df=pd.read_csv('creditcard.csv')
Df

In [ ]:
Df.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,41683.000000,41683.000000,41683.000000,41682.000000,41682.000000,41682.000000,41682.000000,41682.000000,41682.000000,41682.000000,...,41682.000000,41682.000000,41682.000000,41682.000000,41682.000000,41682.000000,41682.000000,41682.000000,41682.000000,41682.000000
mean,26208.345609,-0.222499,0.035454,0.711902,0.186781,-0.234719,0.099510,-0.112246,0.044579,0.193708,...,-0.028417,-0.111341,-0.040301,0.007295,0.136259,0.023199,0.008003,0.004864,89.104007,0.002591
std,12721.514541,1.855101,1.599923,1.490121,1.399707,1.387375,1.309662,1.238350,1.209428,1.220092,...,0.744449,0.638250,0.565217,0.593050,0.436860,0.504139,0.387877,0.341927,237.163503,0.050837
min,0.000000,-56.407510,-72.715728,-31.103685,-5.172595,-42.147898,-26.160506,-26.548144,-41.484823,-7.175097,...,-20.262054,-8.593642,-26.751119,-2.836627,-7.495741,-1.438650,-8.567638,-9.617915,0.000000,0.000000
25%,16853.000000,-0.974000,-0.533197,0.229759,-0.719018,-0.842162,-0.638071,-0.598424,-0.151197,-0.568666,...,-0.235257,-0.533250,-0.179117,-0.324744,-0.127717,-0.329707,-0.063304,-0.006814,7.500000,0.000000
50%,31305.000000,-0.240189,0.093834,0.811276,0.185168,-0.272258,-0.156277,-0.073482,0.050183,0.071496,...,-0.074873,-0.085641,-0.051385,0.060759,0.175870,-0.064901,0.008653,0.021615,24.000000,0.000000
75%,36289.000000,1.159396,0.740067,1.443056,1.069459,0.293052,0.485883,0.432308,0.318625,0.906980,...,0.102520,0.300633,0.077243,0.400073,0.421184,0.304378,0.085015,0.076216,80.000000,0.000000
max,40784.000000,1.960497,16.713389,4.101716,16.491217,34.801666,22.529298,36.677268,20.007208,10.392889,...,22.614889,5.805795,17.297845,4.014444,5.525093,3.517346,11.135740,33.847808,7879.420000,1.000000


In [ ]:
Df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37722 entries, 0 to 37721
Data columns (total 31 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Time    37722 non-null  int64  
 1   V1      37722 non-null  float64
 2   V2      37722 non-null  float64
 3   V3      37722 non-null  float64
 4   V4      37722 non-null  float64
 5   V5      37722 non-null  float64
 6   V6      37722 non-null  float64
 7   V7      37721 non-null  float64
 8   V8      37721 non-null  float64
 9   V9      37721 non-null  float64
 10  V10     37721 non-null  float64
 11  V11     37721 non-null  float64
 12  V12     37721 non-null  float64
 13  V13     37721 non-null  float64
 14  V14     37721 non-null  float64
 15  V15     37721 non-null  float64
 16  V16     37721 non-null  float64
 17  V17     37721 non-null  float64
 18  V18     37721 non-null  float64
 19  V19     37721 non-null  float64
 20  V20     37721 non-null  float64
 21  V21     37721 non-null  float64
 22

In [ ]:
#missing_Vallues
Df.isna().sum()

,0
Time,0
V1,0
V2,0
V3,1
V4,1
V5,1
V6,1
V7,1
V8,1
V9,1


In [ ]:
# Detect outliers using IQR
outlier_info = {}

for column in Df.columns:
    if column != 'Class':  # Skip the target column (Class)
        Q1 = Df[column].quantile(0.25)
        Q3 = Df[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # Identify outliers
        outliers = Df[(Df[column] < lower_bound) | (Df[column] > upper_bound)]
        outlier_info[column] = {
            "outlier_count": outliers.shape[0],
            "lower_bound": lower_bound,
            "upper_bound": upper_bound,
        }

# Convert outlier summary to DataFrame
outlier_summary = pd.DataFrame(outlier_info).T
outlier_summary.columns = ['Outlier Count', 'Lower Bound', 'Upper Bound']

print(outlier_summary)

        Outlier Count   Lower Bound   Upper Bound
Time              0.0 -12301.000000  65443.000000
V1             1102.0     -4.174094      4.359491
V2             2251.0     -2.443093      2.649963
V3             1396.0     -1.590188      3.263002
V4              503.0     -3.401733      3.752173
V5             2576.0     -2.544984      1.995874
V6             3536.0     -2.324002      2.171814
V7             1419.0     -2.144523      1.978406
V8             3899.0     -0.855930      1.023359
V9              836.0     -2.782135      3.120450
V10            1770.0     -1.945413      1.780257
V11             154.0     -2.816060      3.611369
V12             285.0     -3.844900      3.114184
V13             239.0     -2.788262      3.208071
V14            1297.0     -1.841164      2.351375
V15             730.0     -2.313548      2.638261
V16             899.0     -2.067888      2.131704
V17            1174.0     -1.726496      1.950229
V18            1484.0     -2.023002      1.858363


In [ ]:
target = "Class"
100*Df[target].value_counts()/Df.shape[0]

,count
Class,
0.0,99.738503
1.0,0.259098


In [ ]:
Df = Df.dropna(subset=[target])
X = Df.drop(columns=[target])
y = Df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Baseline Evaluation with Random Oversampling

Before implementing GANs to augment the dataset, it is essential to establish a **baseline performance** using a simple and widely-used resampling technique: **Random Oversampling**.

This step involves:
1. Using **Random Oversampling** to balance the training dataset by duplicating samples from the minority class.
2. Training a **Random Forest Classifier** on the oversampled dataset to predict fraud cases.
3. Evaluating the model's performance on the original, unmodified test set using:
   - **Confusion Matrix**: To analyze prediction outcomes for fraud and non-fraud cases.
   - **Precision, Recall, and F1-Score**: To measure the classifier's effectiveness in detecting fraudulent transactions.

The results obtained here will serve as a benchmark for assessing the **effectiveness of GAN-generated synthetic samples** in improving fraud detection. By comparing the baseline metrics with those achieved using GAN-augmented data, we can quantify the added value of GANs.

In [ ]:
csvm=SVC(kernel='linear')
csvm.fit(X_train, y_train)

yc_pred=csvm.predict(X_test)
accuracy=accuracy_score(y_test, yc_pred)

conf_matrix = confusion_matrix(y_test, yc_pred)
classification_summary = classification_report(y_test, yc_pred)
precision = precision_score(y_test, yc_pred)
recall = recall_score(y_test, yc_pred)
f1 = f1_score(y_test, yc_pred)

print("Accuracy:")
print(accuracy)
print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Summary:")
print(classification_summary)
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

Accuracy:
0.9976010555355643
Confusion Matrix:
[[8302   13]
 [   7   15]]

Classification Summary:
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      8315
         1.0       0.54      0.68      0.60        22

    accuracy                           1.00      8337
   macro avg       0.77      0.84      0.80      8337
weighted avg       1.00      1.00      1.00      8337

Precision: 0.54
Recall: 0.68
F1 Score: 0.60


In [ ]:
# Baseline Evaluation with Random Oversampling & SVM
oversampler = RandomOverSampler(sampling_strategy=0.05, random_state=42)
X_train_oversampled, y_train_oversampled = oversampler.fit_resample(X_train, y_train)

csvm = SVC(kernel='linear', class_weight='balanced', random_state=42)
csvm.fit(X_train, y_train)

yc_pred=csvm.predict(X_test)
accuracy=accuracy_score(y_test, yc_pred)

conf_matrix = confusion_matrix(y_test, yc_pred)
classification_summary = classification_report(y_test, yc_pred)
precision = precision_score(y_test, yc_pred)
recall = recall_score(y_test, yc_pred)
f1 = f1_score(y_test, yc_pred)

print("Accuracy:")
print(accuracy)
print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Summary:")
print(classification_summary)
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:484: FutureWarning: `BaseEstimator._check_n_features` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_n_features` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: FutureWarning: `BaseEstimator._check_feature_names` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_feature_names` instead.
  warnings.warn(


Accuracy:
0.9817680220702891
Confusion Matrix:
[[8164  151]
 [   1   21]]

Classification Summary:
              precision    recall  f1-score   support

         0.0       1.00      0.98      0.99      8315
         1.0       0.12      0.95      0.22        22

    accuracy                           0.98      8337
   macro avg       0.56      0.97      0.60      8337
weighted avg       1.00      0.98      0.99      8337

Precision: 0.12
Recall: 0.95
F1 Score: 0.22


In [ ]:
# Baseline Evaluation with GAN Oversampling & SVM
oversampler = GANDataBalancer(sampling_strategy=0.05, random_state=42, latent_dim=100)
X_train_oversampled, y_train_oversampled = oversampler.fit_resample(X_train, y_train)

csvm=SVC(kernel='linear', class_weight='balanced', random_state=42)
csvm.fit(X_train_oversampled, y_train_oversampled)

yc_pred=csvm.predict(X_test)
accuracy=accuracy_score(y_test, yc_pred)

conf_matrix = confusion_matrix(y_test, yc_pred)
classification_summary = classification_report(y_test, yc_pred)
precision = precision_score(y_test, yc_pred)
recall = recall_score(y_test, yc_pred)
f1 = f1_score(y_test, yc_pred)

print("Accuracy:")
print(accuracy)
print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Summary:")
print(classification_summary)
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 0/1000 | Discriminator Loss: [array(0.33838603, dtype=float32), array(0.796875, dtype=float32)] | Generator Loss: [array(0.33838603, dtype=float32), array(0.33838603, dtype=float32), array(0.796875, dtype=float32)]
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
2/2 ━━━━━━━━━━━━━━━

In [ ]:
# Baseline Evaluation with Random Oversampling & RandomForest
oversampler = RandomOverSampler(sampling_strategy=0.05, random_state=42)
X_train_oversampled, y_train_oversampled = oversampler.fit_resample(X_train, y_train)

model = RandomForestClassifier(random_state=42)
model.fit(X_train_oversampled, y_train_oversampled)

y_pred = model.predict(X_test)

conf_matrix = confusion_matrix(y_test, y_pred)
classification_summary = classification_report(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Summary:")
print(classification_summary)
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:484: FutureWarning: `BaseEstimator._check_n_features` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_n_features` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: FutureWarning: `BaseEstimator._check_feature_names` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_feature_names` instead.
  warnings.warn(


Confusion Matrix:
[[7520    4]
 [   3   18]]

Classification Summary:
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      7524
         1.0       0.82      0.86      0.84        21

    accuracy                           1.00      7545
   macro avg       0.91      0.93      0.92      7545
weighted avg       1.00      1.00      1.00      7545

Precision: 0.82
Recall: 0.86
F1 Score: 0.84


In [ ]:
# Baseline Evaluation with GAN Oversampling & RandomForest
oversampler = GANDataBalancer(sampling_strategy=0.05, random_state=42, latent_dim=100)
X_train_oversampled, y_train_oversampled = oversampler.fit_resample(X=X_train, y=y_train)

model = RandomForestClassifier(random_state=42)
model.fit(X_train_oversampled, y_train_oversampled)

y_pred = model.predict(X_test)

conf_matrix = confusion_matrix(y_test, y_pred)
classification_summary = classification_report(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Summary:")
print(classification_summary)
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step  
Epoch 0/1000 | Discriminator Loss: [array(0.3126893, dtype=float32), array(0.90625, dtype=float32)] | Generator Loss: [array(0.3126893, dtype=float32), array(0.3126893, dtype=float32), array(0.90625, dtype=float32)]
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
2/2 ━━━━━━━━

**Conclusion:**

Both approaches are performing similarly in terms of accuracy and classifying the majority class well. However, GAN-based data augmentation has a slightly better recall and F1 score, indicating it might be more effective at improving the detection of fraud (minority class).

 It seems like the GAN method can be a good alternative to traditional oversampling for your fraud detection task.